# Algorytm k-Średnich (k-Means)



### Czym jest algorytm K-Means?

Algorytm k-Means to popularna metoda klasteryzacji (grupowania) danych, która ma na celu podzielenie zbioru danych na \(k\) grup (klastrów), gdzie każdy punkt danych należy do klastra z najbliższym średnim (centroidem). Jest to algorytm iteracyjny, co oznacza, że powtarza kroki przypisania i aktualizacji, aż do osiągnięcia stabilnych klastrów.

Główne idee:
- Podział zbioru danych na zadaną liczbę \(k\) klastrów.
- Każdy klaster charakteryzuje się swoim centrum (centroidem), które jest średnią wszystkich punktów danych należących do tego klastra.
- Minimalizacja średniego błądu kwantyzacji:
  $$
  \mathcal L = \sum_{j=1}^{k} \sum_{x \in S_j} \|x - \mu_j\|^2
  $$
  - przypisujemy najbliższe centrum
  - ulokowanie centrów (pochodna)
  $$\frac {\partial \mathcal L} {\partial u^{(i)}} = - 2 \sum_{x \in S_j} (x^{(i)} - \mu_j^{(i)}) = 0$$
  $$ \sum_{x \in S_j} x^{(i)} - |S_j| \cdot \mu_j^{(i)} = 0$$
  $$ \mu_j^{(i)} = \frac 1 {|S_j|} \sum_{x \in S_j} x^{(i)}$$
  druga pochodna: $\frac {\partial^2 \mathcal L} {\partial^2 u^{(i)}} = 2 > 0, \frac {\partial^2 \mathcal L} {\partial u^{(i)}\partial u^{(j)}} = 0$, zatem mamy minimum.

Algorytm działa w następujących krokach:
1.  **Inicjalizacja**: Losowo wybierz \(k\) punktów danych jako początkowe centroidy.
2.  **Przypisanie**: Przypisz każdy punkt danych do najbliższego centroidu. Tworzy to \(k\) wstępnych klastrów.
3.  **Aktualizacja**: Przelicz centroid każdego klastra jako średnią wszystkich punktów danych przypisanych do tego klastra.
4.  **Powtarzaj**: Powtarzaj kroki 2 i 3, aż do momentu, gdy centroidy przestaną się znacząco zmieniać, lub zostanie osiągnięta maksymalna liczba iteracji.

## Pseudokod algorytmu k-means


**Input:**  
Zbiór danych $X = \{x_1, ..., x_n\}$, liczba klastrów $k$

**Output:**  
Zbiór klastrów $S = \{S_1, ..., S_k\}$ oraz centroidy $\mu = \{\mu_1, ..., \mu_k\}$

---

- Zainicjalizuj $k$ centroidów $\mu_1, ..., \mu_k$ (np. losowo z $X$).
- **Powtarzaj:**
   - **// --- Krok przypisania ---**
     1. Wyczyść wszystkie klastry: $S_j = \emptyset$ dla $j = 1, ..., k$
     2. Dla każdego $i = 1, ..., n$:
        - Znajdź najbliższy centroid dla $x_i$:  
          $$j = \underset{j \in \{1, ..., k\}}{\arg\min} \|x_i - \mu_j\|^2$$
        - Przypisz $x\_i$ do klastra $S\_{j}$ poprzez
          $$S\_{j} = S\_{j} \cup \\{{x\_i\\}}$$
   - **// --- Krok aktualizacji ---**
     3. Zapisz stare centroidy:  
        $\mu_j^{\text{old}} \leftarrow \mu_j$ dla $j = 1, ..., k$
     4. Dla każdego $j = 1, ..., k$:  
        - Przelicz centroid $\mu_j$ jako średnią elementów klastra:  
          $\mu_j = \frac{1}{|S_j|} \sum_{x \in S_j} x$
- **Przerwij jeśli:** centroidy się nie zmieniają (tzn. $\mu_j = \mu_j^{\text{old}}$ dla wszystkich $j$) lub została wykonana maksymalna liczba iteracji.


In [ ]:
# TODO: wizualizacja uczenia kMeans

# Klasteryzacja na przykładzie zbioru danych `MNIST`

## Przygotowanie środowiska



In [ ]:
%pip install scikit-learn umap-learn matplotlib seaborn

## Wczytanie danych mnist



In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np

# Load MNIST data
mnist = fetch_openml(name='mnist_784', version=1)
X, y = mnist.data, mnist.target

# Convert to numpy arrays if not already
if not isinstance(X, np.ndarray):
    X = X.to_numpy()
if not isinstance(y, np.ndarray):
    y = y.to_numpy()

print("MNIST data loaded successfully.")
print(f"Shape of features (X): {X.shape}")
print(f"Shape of labels (y): {y.shape}")

In [ ]:
import matplotlib.pyplot as plt

# Function to display a single digit
def display_digit(digit_data, title="Digit"):
    # Reshape the 784-dimensional vector to a 28x28 image
    image = digit_data.reshape(28, 28)
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')

# Display a few example digits
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    display_digit(X[i], title=f"Label: {y[i]}")

plt.tight_layout()
plt.show()

## Redukcja wymiarowości przy użyciu UMAP

In [ ]:
from umap import UMAP
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Create a UMAP instance
# Using default parameters first, can experiment with n_neighbors and min_dist later
# Setting random_state for reproducibility
reducer = UMAP(n_components=2, random_state=42)
# reducer = TSNE(n_components=2, random_state=42)
# reducer = PCA(n_components=2, random_state=42)

# Apply dimensionality reduction to the data
X_reduced = reducer.fit_transform(X)

# Display the shape of the reduced data
print(f"Shape of reduced data (X_reduced): {X_reduced.shape}")

## Zastosowanie algorytmu k-means

In [ ]:
from sklearn.cluster import KMeans

# Create a KMeans instance
# Set n_clusters to 10 and random_state to 42 as instructed
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10) # Added n_init for clarity and to avoid a future warning

# Fit the KMeans model to the UMAP-reduced data and get the cluster labels
kmeans_labels = kmeans.fit_predict(X_reduced)

# Display the first few cluster labels
print("First 20 KMeans cluster labels:")
print(kmeans_labels[:20])

# Display the shape of the labels array
print(f"Shape of KMeans labels array: {kmeans_labels.shape}")

## Wizualizacja wyników


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set a style for the plot
plt.style.use('seaborn-v0_8-whitegrid')

# Create the scatter plot
plt.figure(figsize=(12, 10))

# Create the scatter plot
# Use the cluster label for color
scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1],
                      c=kmeans_labels,
                      cmap='rainbow', # Use a colormap
                      alpha=0.6,
                      s=20) # Adjust size of points

# Add titles and labels
plt.title('Wyniki klasteryzacji K-Means danych MNIST zredukowanych przez UMAP', fontsize=16)
plt.xlabel('Komponent UMAP 1', fontsize=12)
plt.ylabel('Komponent UMAP 2', fontsize=12)

# Create colorbar with cluster labels
# cbar = plt.colorbar(scatter)
# cbar.set_label('Etykieta Klastra', fontsize=12)
# Set integer ticks for the colorbar
# cbar.set_ticks(np.linspace(0, 9, 10) + 0.5) # Center ticks between colors
# cbar.set_ticklabels(range(10))


# Add text labels for some representative points from each original digit to show the overlap
# To avoid cluttering, we'll sample a few points for each digit
from sklearn.utils import resample
for digit in sorted(np.unique(y)):
    # Select indices corresponding to the current digit
    digit_indices = (y == str(digit))

    # Randomly sample a few indices to label
    if np.sum(digit_indices) > 0: # Ensure there are points for this digit
        sample_indices = resample(np.where(digit_indices)[0], replace=False, n_samples=min(10, np.sum(digit_indices)), random_state=42) # Sample up to 10 points

        # Add text annotation for the sampled points
        for i in sample_indices:
            plt.text(X_reduced[i, 0], X_reduced[i, 1], y[i], fontsize=9, ha='center', va='center')


# Display the plot
plt.show()

# Ćwiczenie (Fashion MNIST)

Powtórz powyższe ćwiczenia na zbiorze danych Fashion MNIST